## `genalog.generation` module:

This module is responsible for:
- Analog document generation

There are two important concepts to be familiar with:
1. **Template** - controls the layout of the document (i.e. font, langauge, position of the content, etc)
1. **Content** - items to be used to fill the template (i.e. text, images, tables, lists, etc)

We are using a HTML templating engine (**Jinja2**) to build our html templates, and a html-pdf converter (**Weasyprint**) to print the html as a pdf or an image.


In [ ]:
from genalog.generation.document import DocumentGenerator
from genalog.generation.content import CompositeContent, ContentType

with open("sample/generation/example.txt", 'r') as f:
    text = f.read()

print(text[:1000])

## `genalog.generation.content` Submodule

This module is used to initialize a `Content` object for populating content into a template. It has the following classes:
1. **ContentType** - an enumeration dictating the supported content type (ex. ContentType.PARAGRAPH, ContentType.TITLE, ContentType.COMPOSITE)
1. **Content** - a base class for inheritance. All `Content` classes should be printable, iterable and indexable.
1. **Paragraph** - a class inherited from `Content`. It represents a paragraph block.
1. **Title** - a class inherited from `Content`. It represents a title of a block.
1. **CompositeContent** - a class inherited from `Content`. It is a composite class designed to hold a hybrid collection of extended `Content` classes in the order they would populate the template. It is the base class used to initalize a document.

In [ ]:
# Initialize Content Object
paragraphs = text.split('\n\n')
content_types = [ContentType.PARAGRAPH] * len(paragraphs)
content = CompositeContent(paragraphs, content_types)

print(f"Object represention: {repr(content)}\n")
# you can also print the content object like a string
print(f"Transparent string: {content}"[:400])

In [ ]:
# Or iterate the content obj like an array
for paragraph in content:
    print(paragraph[:100])

## `genalog.generation.document` Submodule

This module initializes a document by loading the html template and setting up the document styles. It has two classes:
1. **Document** - given a template and the content, this class populates the content into the template and controls the document style. It can print the document as PDF or PNG.
1. **DocumentGenerator** - a factory class that can generate `Document` classes. This will be the primary object used to generate documents.

In [ ]:
# Initalize DocumentGenerator
# The default setting allow to use the built-in templates from the `genalog` package
default_generator = DocumentGenerator()

print(f"Available default templates: {default_generator.template_list}")
print(f"Default styles to generate: {default_generator.styles_to_generate}")

In [ ]:
from IPython.core.display import Image, display

# Select specific template, content and create the generator
doc_gen = default_generator.create_generator(content, ['text_block.html.jinja']) 
# we will use the `content` object initialized from above 

for doc in doc_gen:
    print(doc.styles)
    print(doc.template.name)
    image_byte = doc.render_png(resolution=100)
    display(Image(image_byte))

## Alternative settings:

Apart from the default behaviors, you can customize the tool to your need:
1. Load custom templates
1. Add custom style configurations
1. Save individual pages as separate PNG file

In [ ]:
# Load custom templates in local file path:
custom_document_generator = DocumentGenerator(template_path="../genalog/generation/templates")

print(custom_document_generator.template_list)

In [ ]:
# Create new content with titles and paragraphs
sections = []
section_content_types = []
for index, p in enumerate(paragraphs):
    sections.append(f"Section {index}:")
    section_content_types.append(ContentType.TITLE)
    sections.append(p)
    section_content_types.append(ContentType.PARAGRAPH)

titled_content = CompositeContent(sections, section_content_types)

for c in titled_content:
    print(c[:200])

In [ ]:
# List the available tempaltes:
print(default_generator.template_list)

In [ ]:
# Set new styles to generate
new_style_combinations = {
    "hyphenate": [True],
    "font_size": ["11px"],
    "font_family": ["Times"],
    "text_align": ["justify"]
}
default_generator.set_styles_to_generate(new_style_combinations)
print(f"Styles to generate: {default_generator.styles_to_generate}")

In [ ]:
from IPython.core.display import Image, display
doc_gen = default_generator.create_generator(titled_content, ["columns.html.jinja", "letter.html.jinja"])

for doc in doc_gen:
    print(doc.styles)
    print(doc.template.name)
    # Save with different resolution
    image_byte = doc.render_png(resolution=90)
    display(Image(image_byte))

In [ ]:
doc_gen = default_generator.create_generator(titled_content, ["columns.html.jinja", "letter.html.jinja"])
default_generator.set_styles_to_generate(new_style_combinations)
# Or you can save the document as a separate file
for doc in doc_gen:
    template_name = doc.template.name.replace(".html.jinja", "")
    pdf_name = "sample/generation/" + template_name + "_" + doc.styles["font_family"] + "_" + doc.styles["font_size"]  + ".pdf"
    png_name = pdf_name.replace(".pdf", ".png")
    doc.render_pdf(target=pdf_name, zoom=2)
    doc.render_png(target=png_name, resolution=100)

In [ ]:
# Save as separate PNG files
doc_gen = default_generator.create_generator(content, ["text_block.html.jinja"])
default_generator.set_styles_to_generate(new_style_combinations)
# Configure image resolution in dots per inch (dpi)
resolution = 300
for doc in doc_gen:
    template_name = doc.template.name.replace(".html.jinja", "")
    png_name = "sample/generation/" + template_name + "_" + doc.styles["font_family"] + "_" + doc.styles["font_size"] + ".png"
    doc.render_png(target=png_name, split_pages=True, resolution=resolution)